[View in Colaboratory](https://colab.research.google.com/github/spongeclipper/korruption/blob/master/PAC2PAC.ipynb)

In [0]:
import collections, os, pandas as pd, urllib, zipfile
from datetime import datetime

In [0]:
#@title Download the pac2pac donations list for 2017-2018.

# Download the PAC contributions file, and unzip it.
urllib.request.urlretrieve('https://www.fec.gov/files/bulk-downloads/2018/oth18.zip', 'oth18.zip')
with zipfile.ZipFile('oth18.zip') as zip_ref:
  zip_ref.extractall('.')
  
# Download the PAC descriptions file.
urllib.request.urlretrieve('https://www.fec.gov/files/bulk-downloads/2018/cm18.zip', 'cm18.zip')
with zipfile.ZipFile('cm18.zip') as zip_ref:
  zip_ref.extractall('.')

In [39]:
# Populate 'donations' with the list of transactions.  
Donation = collections.namedtuple('Donation',
                                  ['Recipient',
                                   'Donor',
                                   'Amount',
                                   'Election',
                                   'Date',
                                  ])
Pac = collections.namedtuple('Pac',
                             ['ID',
                              'Name',
                              'Zip',
                              'State',
                              'Treasurer',
                              'Type',
                             ])

ds = []  # Donations
ps = []  # Pacs

knownIDs = set()  # Used below to dedupe records.

# Parse the PAC descriptions:
with open('cm.txt', 'r') as raw:
  for l in raw.readlines():
    parts = l.strip().split('|')

    ps.append(Pac(
        ID=parts[0],
        Name=parts[1],
        Zip=parts[7],
        State=parts[6],
        Treasurer=parts[2],
        Type=parts[9],
    ))
    knownIDs.add(parts[0])
    
# Parse the pac2pac contributions:
with open('itoth.txt', 'r') as raw:
  for l in raw.readlines():
    parts = l.strip().split('|')
    
    if not parts[15]:
      continue  # This is not a donation?
    try:
      dt = datetime.strptime(parts[13], '%m%d%Y')
    except ValueError:
      pass
  
    ds.append(Donation(
        Recipient=parts[0],
        Donor=parts[15],
        Amount=int(parts[14]),
        Election=parts[3],
        Date=dt,
    ))
    # The "Committed" data we downloaded above doesn't contain info on every PAC,
    # though I'm not sure why. So if we're missing a PAC we can infer data
    # (like name and ZIP) from this file.
    if parts[15] not in knownIDs:
      ps.append(Pac(
        ID=parts[15],
        Name=parts[7],
        Zip=parts[10],
        State=parts[9],
        Treasurer=None,
        Type=parts[6],
      ))
      knownIDs.add(parts[15])
#############
# Data frames
#############
donations = pd.DataFrame.from_records(ds, columns=Donation._fields)
pacs = pd.DataFrame.from_records(ps, columns=Pac._fields).set_index('ID')

print('Parsed %d donations and %d PACs' % (len(donations), len(pacs)))

Parsed 308048 donations and 20437 PACs


In [40]:
#@title Who's the biggest donor?
donations.loc[:, ['Donor', 'Amount']].groupby(
    'Donor').sum().join(pacs).sort_values(
    'Amount', ascending=False)

,Amount,Name,Zip,State,Treasurer,Type
C00075820,50323410,NRCC,20003,DC,"DAVIS, KEITH A.",Y
C00545947,46007785,TEAM RYAN,20003,DC,"KILGORE, PAUL",N
C00002931,33045433,NATIONAL REPUBLICAN CONGRESSIONAL COMMITTEE BU...,20003,DC,None,PTY
C00000935,32961812,DCCC,200034024,DC,"SENA, DANIEL",Y
C00003418,32312588,REPUBLICAN NATIONAL COMMITTEE,20003,DC,"PARKER, ANTHONY W MR",Y
C00618371,30975445,TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE,01915,MA,"CRATE, BRADLEY T. MR.",N
C00010603,29853560,DNC SERVICES CORP./DEM. NAT'L COMMITTEE,20003,DC,WILLIAM Q. DERROUGH,Y
C00027466,24853451,NRSC,20002,DC,"DAVIS, KEITH A",Y
C00580100,24761794,"DONALD J. TRUMP FOR PRESIDENT, INC.",10022,NY,"CRATE, BRADLEY T. MR.",P
C00330894,18506281,"RYAN FOR CONGRESS, INC.",535471488,WI,"MAIR, PAUL",H


In [42]:
#@title Who's the biggest recipient?
donations.loc[:, ['Recipient', 'Amount']].groupby(
    'Recipient').sum().join(pacs).sort_values(
    'Amount', ascending=False)

,Amount,Name,Zip,State,Treasurer,Type
C00075820,99775476,NRCC,20003,DC,"DAVIS, KEITH A.",Y
C00000935,49528851,DCCC,200034024,DC,"SENA, DANIEL",Y
C00545947,49482693,TEAM RYAN,20003,DC,"KILGORE, PAUL",N
C00003418,34620706,REPUBLICAN NATIONAL COMMITTEE,20003,DC,"PARKER, ANTHONY W MR",Y
C00580100,31519295,"DONALD J. TRUMP FOR PRESIDENT, INC.",10022,NY,"CRATE, BRADLEY T. MR.",P
C00010603,27915723,DNC SERVICES CORP./DEM. NAT'L COMMITTEE,20003,DC,WILLIAM Q. DERROUGH,Y
C00027466,26964070,NRSC,20002,DC,"DAVIS, KEITH A",Y
C00618371,24497927,TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE,01915,MA,"CRATE, BRADLEY T. MR.",N
C00484642,23779417,SMP,20005,DC,"LAMBE, REBECCA",O
C00042366,20874831,DSCC,20002,DC,ALLISON WRIGHT,Y


In [43]:
#@title Can we find "passthrough" PACs by looking for those with mostly-matched negative and positive value transactions?
pacTransactions = collections.defaultdict(list)
for d in donations.iterrows():
  # Recipient records a balance, and donor a loss
  pacTransactions[d[1].Recipient].append(d[1].Amount)
  pacTransactions[d[1].Donor].append(0-d[1].Amount)
  
matchedVsUnmatched = collections.defaultdict(tuple)  # let's go through each PAC and count (matched, unmatched)
for pac, transactions in pacTransactions.items():
  matched = 0
  unmatched = 0
  while transactions:
    x = transactions[0]
    if 0-x in transactions[1:]:
      matched+=1
      ts = transactions[1:]
      idx = ts.index(0-x)
      ts = ts[:idx] + ts[idx+1:]
      transactions = ts
    else:
      unmatched+=1
      transactions = transactions[1:]
  matchedVsUnmatched[pac] = (matched, unmatched)

# Sort by matched - unmatched
s = sorted(matchedVsUnmatched.items(), key=lambda x: x[1][0]-x[1][1])
s.reverse()
s

[('C00010603', (1920, 660)),
 ('C00075820', (1800, 584)),
 ('C00027466', (1078, 102)),
 ('C00042366', (1328, 360)),
 ('C00000935', (1898, 1302)),
 ('C00409003', (435, 67)),
 ('C00409730', (531, 196)),
 ('C00387464', (385, 73)),
 ('C00495887', (324, 38)),
 ('C00525543', (292, 23)),
 ('C00388421', (325, 68)),
 ('C00003418', (629, 376)),
 ('C00344648', (256, 9)),
 ('C00491654', (263, 39)),
 ('C00348607', (270, 53)),
 ('C00147512', (250, 35)),
 ('C00365536', (308, 95)),
 ('C00235655', (250, 44)),
 ('C00394957', (554, 354)),
 ('C00436022', (215, 20)),
 ('C00193342', (224, 35)),
 ('C00361956', (192, 13)),
 ('C00440032', (202, 27)),
 ('C00390674', (181, 8)),
 ('C00658476', (254, 82)),
 ('C00536607', (228, 61)),
 ('C00548651', (215, 49)),
 ('C00459123', (277, 114)),
 ('C00458877', (291, 141)),
 ('C00480228', (160, 13)),
 ('C00442368', (166, 20)),
 ('C00445379', (245, 100)),
 ('C00399196', (253, 109)),
 ('C00326082', (176, 32)),
 ('C00305318', (273, 129)),
 ('C00330894', (691, 549)),
 ('C005713